In [ ]:
import pandas as pd
import numpy as np

In [ ]:
print('HieuVM')

HieuVM


In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1IkAlq6omD87awKdQiD4tlZB4W92VOLWdqRFhN1-LVK4/export?format=xlsx'
df = pd.read_excel(url)

In [ ]:
df.tail()

,CUst ID,JOIN% DATE Org,JOIN% DATE,Unnamed: 3,mobiles,FLL NAM
95,96,20190816,20190816,NaN,84334555439,Ron Grollimund
96,97,2017-08-22 00:00:1503334800,09/02/19,aa,84972812359,Daniel Bentley
97,98,2018-02-17 00:00:1518800400,2018-02-17 00:00:00,aa,84865784802,Eugene Brown
98,99,20190612,20190612,aa,841221343238,elMER milLER
99,100,20190822,20190822,NaN,1264626143,Frances Holland


In [ ]:
print(df.shape)

(100, 6)


# 3.1  Clean the names of columns to lowercase separated by “_”, remove any empty column if necessary.

In [ ]:
# Check empty column
df.isnull().sum()

CUst ID              0
  JOIN% DATE Org     0
  JOIN% DATE         0
Unnamed: 3          97
mobiles              0
  FLL NAM            0
dtype: int64

In [ ]:
# Remove empty column
df = df.drop("Unnamed: 3", axis=1)

In [ ]:
df.head()

,CUst ID,JOIN% DATE Org,JOIN% DATE,mobiles,FLL NAM
0,1,08/07/19,08/07/19,84333605993,mIChAel mIcHALek
1,2,09/10/18,09/10/18,973444062,Andrew Jimenez
2,3,20190630,20190630,338262954,Ann Gow
3,4,2019-09-06 00:00:1567702800,2019-09-06 00:00:00,84966068026,James Chen
4,5,20170812,20170812,84767065885,Dollie Martinez


In [ ]:
# Clean the names of columns to lowercase separated by “_"

df.columns = df.columns.str.replace("%","")
df.columns = df.columns.str.lstrip()
df.rename(columns={'CUst ID':'cust_id','JOIN DATE':'join_date','FLL NAM':'full_name'}, inplace=True)

In [ ]:
print(df.shape)
print(df.columns)

(100, 5)
Index(['cust_id', 'JOIN DATE Org', 'join_date', 'mobiles', 'full_name'], dtype='object')


# 3.2 Change the name column to the title case (e.g: Jason Mraz).

In [ ]:
# Change all character to UPPERCASE
df['full_name'] = df['full_name'].str.upper()
# Change all character to Proper Case
df['full_name'] = df.full_name.str.title()
# Validate
df['full_name']

0     Michael Michalek
1       Andrew Jimenez
2              Ann Gow
3           James Chen
4      Dollie Martinez
            ...       
95      Ron Grollimund
96      Daniel Bentley
97        Eugene Brown
98        Elmer Miller
99     Frances Holland
Name: full_name, Length: 100, dtype: object

# 3.3 Make a new “email” column with form: {last_name}.{first_name}.{id}@yourcompany.com

In [ ]:
# Creating email with form: {last_name}.{first_name}.{id}@yourcompany.com

df['email'] = df.full_name.str.lower().replace(" ",".") + "." + df.cust_id.astype(str)+ "@yourcompany.com"

In [ ]:
df.head()

,cust_id,JOIN DATE Org,join_date,mobiles,full_name,email
0,1,08/07/19,08/07/19,84333605993,Michael Michalek,michael michalek.1@yourcompany.com
1,2,09/10/18,09/10/18,973444062,Andrew Jimenez,andrew jimenez.2@yourcompany.com
2,3,20190630,20190630,338262954,Ann Gow,ann gow.3@yourcompany.com
3,4,2019-09-06 00:00:1567702800,2019-09-06 00:00:00,84966068026,James Chen,james chen.4@yourcompany.com
4,5,20170812,20170812,84767065885,Dollie Martinez,dollie martinez.5@yourcompany.com


## 3.4 Change the date column to the same format ‘YYYY-MM-DD’

In [ ]:
df['join_date']=pd.to_datetime(df['join_date'],errors='coerce')
unique=pd.to_datetime(df['join_date'], format='%Y-%m-%d %H:%M:%S%f',errors='coerce').dt.strftime('%Y-%m-%d')
df.loc[df['join_date'].isnull(),'join_date']=unique
df['join_date']=pd.to_datetime(df['join_date'],errors='coerce')

In [ ]:
df.head()

,cust_id,JOIN DATE Org,join_date,mobiles,full_name,email
0,1,08/07/19,2019-08-07,84333605993,Michael Michalek,michael michalek.1@yourcompany.com
1,2,09/10/18,2018-09-10,973444062,Andrew Jimenez,andrew jimenez.2@yourcompany.com
2,3,20190630,2019-06-30,338262954,Ann Gow,ann gow.3@yourcompany.com
3,4,2019-09-06 00:00:1567702800,2019-09-06,84966068026,James Chen,james chen.4@yourcompany.com
4,5,20170812,2017-08-12,84767065885,Dollie Martinez,dollie martinez.5@yourcompany.com


# 3.5 Change the phone number column to the format **“84……”**

In [ ]:
# Change data type of Mobiles col. to string and replace "+" with ""
df['mobiles'] = df['mobiles'].astype(str).str.replace('+', '')
df.loc[~df['mobiles'].str.startswith('84'), 'mobiles'] = '84' + df[~df['mobiles'].str.startswith('84')]['mobiles']

# Or I can handle it in Excel

# Validate
df['mobiles']

0      84333605993
1      84973444062
2      84338262954
3      84966068026
4      84767065885
          ...     
95     84334555439
96     84972812359
97     84865784802
98    841221343238
99    841264626143
Name: mobiles, Length: 100, dtype: object

# 3.6 Find any duplicated ID and remove those who join later

In [ ]:
## Sorting by join date ascending and then dropping the duplicate values by keeping the lowest date

df = df.sort_values(by='join_date', ascending=True)
df = df.drop_duplicates(subset='cust_id', keep="last")

In [ ]:
df.head()

,cust_id,JOIN DATE Org,join_date,mobiles,full_name,email
35,36,02/10/17,2017-02-10,84339661824,Gregory Miele,gregory miele.36@yourcompany.com
37,38,20170802,2017-08-02,84344179399,Melvin Pigg,melvin pigg.38@yourcompany.com
25,26,20170807,2017-08-07,84966042581,Charles Garnand,charles garnand.26@yourcompany.com
4,5,20170812,2017-08-12,84767065885,Dollie Martinez,dollie martinez.5@yourcompany.com
38,39,20171010,2017-10-10,84335429970,Mark Faria,mark faria.39@yourcompany.com


In [ ]:
df.shape

(90, 6)

# 3.7 Filter those who join since 2019 and export to a csv file, delimited by “|”, file name “emp_{report_date}.csv” with report_date = today.


In [ ]:
import time

report_date = time.strftime("%Y%m%d")
export_filename = "emp_" + report_date + ".csv"

In [ ]:
filter = df[(df['join_date']> "2019-01-01")].to_csv(export_filename,sep ='|',index=False)